In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle
from utilities import *

In [2]:
#Load config
config = loadData("config")
dataSetSize = config["dataSetSize"]
testSetBenignSize = config["testSetBenignSize"]
validationSetSize = config["validationSetSize"]
trainingSetSize = config["trainingSetSize"]
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]
adversarialTestSetSize = config["adversarialTestSetSize"]

In [3]:
thresholds = loadData("thresholds")

### PCA

In [4]:
adversarialSet = loadData("adversarial_data_set_pca")
advSetSize = len(adversarialSet)
advTestSet = adversarialSet[advSetSize - adversarialTestSetSize:]
_,benignTestSet,_ = getReshapedDataSet(loadData("normalized_data_set"), "PCA")
test_labels = np.concatenate((np.ones(len(advTestSet)),np.zeros(len(benignTestSet))))
testSet = np.concatenate((getReshapedTestSet(advTestSet, "PCA"),benignTestSet))

precisions = []
recalls = []
model = load_model('Trained_Model/pca_adv.h5')
predicted = model.predict(testSet)
mse = (np.square(testSet - predicted)).mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision, recall, minPositiveMSE, maxNegativeMSE = rankedPrecisionAndRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
PCAPrecisions_domain = precisions
thre = calculateThreshold(minPositiveMSE,maxNegativeMSE)
thresholds["PCA_adv"] = thre
saveData(thresholds,"thresholds")
precisionThre, recallThre = rankedPrecisionAndRecallWithThreshold(mse_label,thre)

print("Precision without threshold is: ", precision)
print("Recall without threshold is: ", recall)
print("minPositiveMSE",minPositiveMSE)
print("maxNegativeMSE",maxNegativeMSE)
print("Threshold is ", thre)
print("Precision by threshold is: ", precisionThre)
print("Recall by threshold is: ", recallThre)

Precision without threshold is:  1.0
Recall without threshold is:  1.0
minPositiveMSE 0.0032279734131176706
maxNegativeMSE 0.0015371517642618381
Threshold is  0.0017062339291474215
Precision by threshold is:  1.0
Recall by threshold is:  1.0


### Fully Connected

In [22]:
adversarialSet = loadData("adversarial_data_set_fullyConnected")
advSetSize = len(adversarialSet)
advTestSet = adversarialSet[advSetSize - adversarialTestSetSize:]
_,benignTestSet,_ = getReshapedDataSet(loadData("normalized_data_set"), "fullyConnected")
test_labels = np.concatenate((np.ones(len(advTestSet)),np.zeros(len(benignTestSet))))
testSet = np.concatenate((getReshapedTestSet(advTestSet, "fullyConnected"),benignTestSet))

precisions = []
recalls = []
model = load_model('Trained_Model/autoencoder_adv.h5')
predicted = model.predict(testSet)
mse = (np.square(testSet - predicted)).mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision, recall, minPositiveMSE, maxNegativeMSE = rankedPrecisionAndRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
PCAPrecisions_domain = precisions
thre = thresholds["fullyConnected"]
precisionThre, recallThre = rankedPrecisionAndRecallWithThreshold(mse_label,thre)

print("Precision without threshold is: ", precision)
print("Recall without threshold is: ", recall)
print("minPositiveMSE",minPositiveMSE)
print("maxNegativeMSE",maxNegativeMSE)
print("Threshold is ", thre)
print("Precision by threshold is: ", precisionThre)
print("Recall by threshold is: ", recallThre)

Precision without threshold is:  1.0
Recall without threshold is:  1.0
minPositiveMSE 0.003073575349258088
maxNegativeMSE 0.0010811324296685183
Threshold is  0.0013730668853040838
Precision by threshold is:  1.0
Recall by threshold is:  1.0


### 1D Conv

In [23]:
adversarialSet = loadData("adversarial_data_set_conv")
advSetSize = len(adversarialSet)
advTestSet = adversarialSet[advSetSize - adversarialTestSetSize:]
_,benignTestSet,_ = getReshapedDataSet(loadData("normalized_data_set"), "1DConv")
test_labels = np.concatenate((np.ones(len(advTestSet)),np.zeros(len(benignTestSet))))
testSet = np.concatenate((getReshapedTestSet(advTestSet, "1DConv"),benignTestSet))

precisions = []
recalls = []
model = load_model('Trained_Model/conv_adv.h5')
predicted = model.predict(testSet)
mse = (np.square(testSet - predicted)).mean(axis=1)
mse = mse.reshape(len(testSet)) # for conv only
mse_label = np.vstack((mse, test_labels)).T
precision, recall, minPositiveMSE, maxNegativeMSE = rankedPrecisionAndRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
PCAPrecisions_domain = precisions
thre = thresholds["1DConv"]
precisionThre, recallThre = rankedPrecisionAndRecallWithThreshold(mse_label,thre)

print("Precision without threshold is: ", precision)
print("Recall without threshold is: ", recall)
print("minPositiveMSE",minPositiveMSE)
print("maxNegativeMSE",maxNegativeMSE)
print("Threshold is ", thre)
print("Precision by threshold is: ", precisionThre)
print("Recall by threshold is: ", recallThre)

Precision without threshold is:  0.9157509157509157
Recall without threshold is:  0.9157509157509157
minPositiveMSE 0.0022853074886004694
maxNegativeMSE 8.91216968976937e-05
Threshold is  3.693430875407472e-05
Precision by threshold is:  0.9157509157509157
Recall by threshold is:  1.0


### LSTM

In [24]:
adversarialSet = loadData("adversarial_data_set_lstm")
advSetSize = len(adversarialSet)
advTestSet = adversarialSet[advSetSize - adversarialTestSetSize:]
_,benignTestSet,_ = getReshapedDataSet(loadData("normalized_data_set"), "LSTM")
test_labels = np.concatenate((np.ones(len(advTestSet)),np.zeros(len(benignTestSet))))
testSet = np.concatenate((getReshapedTestSet(advTestSet, "LSTM"),benignTestSet))

precisions = []
recalls = []
model = load_model('Trained_Model/lstm_adv.h5')
predicted = model.predict(testSet)
mse = (np.square(testSet - predicted)).mean(axis=2).mean(axis=1)
mse_label = np.vstack((mse, test_labels)).T
precision, recall, minPositiveMSE, maxNegativeMSE = rankedPrecisionAndRecall(mse_label)
precisions.append(precision)
recalls.append(recall)
PCAPrecisions_domain = precisions
thre = thresholds["LSTM"]
precisionThre, recallThre = rankedPrecisionAndRecallWithThreshold(mse_label,thre)

print("Precision without threshold is: ", precision)
print("Recall without threshold is: ", recall)
print("minPositiveMSE",minPositiveMSE)
print("maxNegativeMSE",maxNegativeMSE)
print("Threshold is ", thre)
print("Precision by threshold is: ", precisionThre)
print("Recall by threshold is: ", recallThre)

Precision without threshold is:  0.15318627450980393
Recall without threshold is:  0.15318627450980393
minPositiveMSE 0.014541888110610808
maxNegativeMSE 0.003002605852733423
Threshold is  1.0687692541864324e-05
Precision by threshold is:  0.15318627450980393
Recall by threshold is:  1.0


In [16]:
advTestSet.shape

(250, 1, 200, 9)

In [17]:
benignTestSet.shape

(1382, 200, 9)